In [ ]:
import pandas as pd 
import numpy as np
import torch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn

In [391]:

# Load dataset
data = load_breast_cancer()
x = data.data
y = data.target
feature_names = data.feature_names  

df_x = pd.DataFrame(data=x, columns=feature_names)
df_y = pd.DataFrame(data=y, columns=['label'])


In [392]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.33, random_state=42
)


In [393]:
df_x.head(4)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300


In [394]:
df_y.sample(5)


,label
542,1
246,1
216,1
557,1
309,1


In [395]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     df_x, df_y, test_size=0.33, random_state=42)


In [396]:

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [397]:

# Convert to tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train.values).float().view(-1, 1)
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test.values).float().view(-1, 1)


In [ ]:


# Custom Simple NN (Logistic Regression)
class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float32, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float32, requires_grad=True)

    def forward(self, x):
        z = torch.matmul(x, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y_pred, y_true):
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
        loss = -(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred)).mean()
        return loss



In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features,3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(3,1)
        self.sigmoid = nn.Sigmoid()



    def forward(self, features):
        out = self.linear1(features)
        out = torch.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        


        return out


In [ ]:

# Training
learning_rate = 0.01
epochs = 250
model = MySimpleNN(X_train_tensor)


In [400]:

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = model.loss_function(y_pred, y_train_tensor)
    loss.backward()

    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 7.1176
Epoch 5, Loss: 6.8545
Epoch 10, Loss: 6.5750
Epoch 15, Loss: 6.2904
Epoch 20, Loss: 5.9969
Epoch 25, Loss: 5.7031
Epoch 30, Loss: 5.4042
Epoch 35, Loss: 5.0850
Epoch 40, Loss: 4.7657
Epoch 45, Loss: 4.4513
Epoch 50, Loss: 4.1439
Epoch 55, Loss: 3.8367
Epoch 60, Loss: 3.5274
Epoch 65, Loss: 3.2241
Epoch 70, Loss: 2.9308
Epoch 75, Loss: 2.6522
Epoch 80, Loss: 2.3847
Epoch 85, Loss: 2.1375
Epoch 90, Loss: 1.9112
Epoch 95, Loss: 1.7062
Epoch 100, Loss: 1.5222
Epoch 105, Loss: 1.3573
Epoch 110, Loss: 1.2141
Epoch 115, Loss: 1.0889
Epoch 120, Loss: 0.9799
Epoch 125, Loss: 0.8845
Epoch 130, Loss: 0.8011
Epoch 135, Loss: 0.7282
Epoch 140, Loss: 0.6646
Epoch 145, Loss: 0.6090
Epoch 150, Loss: 0.5606
Epoch 155, Loss: 0.5184
Epoch 160, Loss: 0.4817
Epoch 165, Loss: 0.4497
Epoch 170, Loss: 0.4217
Epoch 175, Loss: 0.3972
Epoch 180, Loss: 0.3755
Epoch 185, Loss: 0.3563
Epoch 190, Loss: 0.3393
Epoch 195, Loss: 0.3240
Epoch 200, Loss: 0.3102
Epoch 205, Loss: 0.2978
Epoch 210, Los

In [401]:
with torch.no_grad():
    y_pred=model.forward(X_test_tensor)
    y_pred=(y_pred>0.5).float()
    accuracy = (y_pred== y_test_tensor).float().mean()
    print(f'Accuracy : {accuracy.item()}')

Accuracy : 0.9414893388748169
